In [ ]:
import tensorflow as tf
import os 

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
import cv2
import PIL

In [ ]:
data_dir = 'data'

In [ ]:
image_exts = ['jpeg','jpg','bmg','png']

In [ ]:
for image_class in os.listdir(data_dir):
    for image in os.listdir(os.path.join(data_dir, image_class)):
        image_path = os.path.join(data_dir, image_class, image)
        try:
            img = cv2.imread(image_path)
            tip = PIL.what(image_path)
            if tip not in image_exts:
                print('Image not in exr list {}'.format(image_path))
                os.remove(image_path)
        except Exception as e:
            print('Issue with image {}'.format(image_path))
            

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
tf.keras.utils.image_dataset_from_directory??

In [33]:

# Load the dataset from directory
data = tf.keras.utils.image_dataset_from_directory(
    'data',  # Replace with your actual data directory
    labels='inferred',  # Automatically infer labels
    label_mode='int',  # Use integer labels
    batch_size=32,  # Size of the batches of data
    image_size=(256, 256),  # Resize images to 256x256
    shuffle=True,  # Shuffle the data
    seed=123  # Random seed for reproducibility
)

# Example to iterate through the dataset and display one batch shape
for images, labels in data.take(1):
    print(images.shape, labels.shape)

NameError: name 'verbose' is not defined

In [ ]:
data_iterator = data.as_numpy_iterator()